# Handwritten Digit recognizer

The purpose of this model is to develop a  Convolutional Neural Network for recognizing handwritten digits


### Import statements

In [13]:
import tensorflow as tf

from keras.preprocessing import image

from tensorflow.python.keras.callbacks import ReduceLROnPlateau


## The data preparation


#### Data visualisation

In [14]:
#TODO:

### Data Validation

Validate the dataset using the data_validator method

In [15]:
from src.modules.data_validator import  data_validator
training_path = "C:\\Users\\idris\\OneDrive\\Bureau\\Study\\s6\\DATA\\digtis\\mnist_png\\training\\"
testing_path =  "C:\\Users\\idris\\OneDrive\\Bureau\\Study\\s6\\DATA\\digtis\\mnist_png\\testing\\"
for i in range(10):
    data_validator(training_path+ str(i))
    data_validator(testing_path+ str(i))

### Generate directory to read images from & Data augmentation

#### Normalization

In [16]:
train_datagen = image.ImageDataGenerator(rescale=1.0/255.) #normalisation
test_datagen = image.ImageDataGenerator(rescale=1.0/255.)


#### Generate and set directory to read images from with data augnemtation

To avoid overfitting we will alter our dataset.
We chose :

    - Random rotation by 10° of some training image
    - Brightness_range from 30 to 70 : The user will take picture so we will adapt our model

    -Shift image to make them not in the center
        - Randomly shift images vertically
        - Randomly shift images horizontally


In [17]:
#object of class ImageDataGenerator with the recale property
train_generator = train_datagen.flow_from_directory(training_path,batch_size=250,class_mode='sparse',target_size=(28, 28),rotation_range=10,brightness_range= [30,70], zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.3,
        height_shift_range=0.3, )

test_generator = test_datagen.flow_from_directory(testing_path, batch_size=250,class_mode='sparse',target_size=(28, 28))


TypeError: ImageDataGenerator.flow_from_directory() got an unexpected keyword argument 'rotation_range'

test_generator & train_generator are directory to read images from.
Each subdirectory (digits directory) in this directory will be considered to contain images from one class.

In [ ]:
print(test_generator.directory)
print(train_generator.class_indices)

## The CNN modeling and evaluation

### Model


Model : we have to avoid overfitting

Softmax :  Softmax takes a set of values, and effectively picks the biggest one, so, for example, if the output of the last layer looks like [0.1, 0.1, 0.05, 0.1, 9.5, 0.1, 0.05, 0.05, 0.05], it saves you from fishing through it looking for the biggest value, and turns it into [0,0,0,0,1,0,0,0,0] -- The goal is to save a lot of coding!
activation layer : tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
10 : because we have ten classes

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.Keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.Keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.Keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='softmax')
])

#### compile the model
Following the results got by  https://github.com/sanghvirajit/Feedforward_Neural_Network
We make the choice of the RMSprop optimizer


In [ ]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

#### Train the model

Reduce learning rate when a metric has stopped improving,we will gain in time while fitting the model by converging faster to the global minimum by decreasing the learning rate.
Set a learning rate annealer. we chose to decrease by 75%

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=5, # number of epochs with no improvement after which learning rate will be reduced.
                                            verbose=1, #update messages
                                            factor=0.25, #new_lr = lr * factor`.
                                            min_lr=0.00001 # minimum learning rate
                                            )

In [ ]:
history = model.fit(train_generator, epochs=150, steps_per_epoch=30,
        validation_data=test_generator, validation_steps=6)

## Model Evaluation

In [ ]:
##TODO